In [44]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

In [45]:
households = pd.read_csv("Synthetic/households.csv")
persons = pd.read_csv("Synthetic/persons.csv")
trips = pd.read_csv("Synthetic/final_trips.csv")
tours = pd.read_csv("Synthetic/final_tours.csv")
zones = pd.read_csv("Synthetic/mgra15_based_input2022.csv")
pd.set_option('display.max_columns', None)

C:\Users\RyanS\AppData\Local\Temp\ipykernel_28544\2040424636.py:3: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("Synthetic/final_trips.csv")
C:\Users\RyanS\AppData\Local\Temp\ipykernel_28544\2040424636.py:4: DtypeWarning: Columns (15,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  tours = pd.read_csv("Synthetic/final_tours.csv")


In [46]:
households = households.rename(columns={'hhid': 'household_id'}).drop(['Unnamed: 0.4','Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0'], axis=1)
persons = persons.rename(columns={'perid': 'person_id', 'hhid': 'household_id'}).drop(['Unnamed: 0.4','Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0'], axis=1)
zones = zones.drop('Unnamed: 0', axis=1)

In [47]:
feature_df = pd.merge(tours, persons, how='left', on=['person_id', 'household_id'])
feature_df = pd.merge(feature_df, households[['household_id', 'taz', 'hinc']], how='left', on='household_id')
feature_df = pd.merge(trips[['trip_id', 'tour_id', 'purpose']], feature_df, how='left', on='tour_id')
feature_df = feature_df.drop(['tdd', 'tour_id_temp', 'destination_logsum', 'vehicle_occup_1', 'vehicle_occup_2', 'vehicle_occup_3.5', 'mode_choice_logsum'], axis=1)
feature_df = feature_df.rename(columns={'purpose': 'trip_purpose', 'start': 'start_time', 'end': 'end_time', 'pemploy': 'employment_status', 'pstudent': 'student_status', 'ptype': 'person_type', 'educ': 'education', 'weeks': 'weeks_worked', 'hours': 'hours_worked', 'rac1p': 'race', 'taz': 'household_taz', 'hinc': 'household_income'})
feature_df = feature_df.drop(['primary_purpose', 'household_serial_no', 'miltary', 'pnum', 'occen5', 'occsoc5', 'indcen', 'hisp', 'version', 'naics2_original_code', 'soc2'], axis=1)
feature_df.head(5)

,trip_id,tour_id,trip_purpose,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,origin,household_id,start_time,end_time,duration,school_esc_outbound,school_esc_inbound,num_escortees,composition,is_external_tour,is_internal_tour,tour_mode,selected_vehicle,atwork_subtour_frequency,parent_tour_id,stop_frequency,age,sex,employment_status,student_status,person_type,education,grade,weeks_worked,hours_worked,race,household_taz,household_income
0,473,59,escort,1,escort,3,3,4,4,non_mandatory,1,5313.0,57.0,1,8.0,8.0,0.0,NaN,NaN,NaN,NaN,False,True,SHARED3,Van_6_Gas,NaN,NaN,0out_0in,34,2,1,3,1,13,0,1,40,1,46,78621
1,477,59,home,1,escort,3,3,4,4,non_mandatory,1,5313.0,57.0,1,8.0,8.0,0.0,NaN,NaN,NaN,NaN,False,True,SHARED3,Van_6_Gas,NaN,NaN,0out_0in,34,2,1,3,1,13,0,1,40,1,46,78621
2,505,63,eatout,2,eatout,1,1,1,1,joint,5,2574.0,57.0,1,30.0,31.0,1.0,NaN,NaN,NaN,children,False,True,WALK,NaN,NaN,NaN,0out_0in,16,2,3,1,6,0,5,0,0,1,46,78621
3,509,63,home,2,eatout,1,1,1,1,joint,5,2574.0,57.0,1,30.0,31.0,1.0,NaN,NaN,NaN,children,False,True,WALK,NaN,NaN,NaN,0out_0in,16,2,3,1,6,0,5,0,0,1,46,78621
4,521,65,othdiscr,1,othdiscr,1,1,3,4,joint,3,3481.0,57.0,1,15.0,17.0,2.0,NaN,NaN,NaN,mixed,False,True,WALK,NaN,NaN,NaN,0out_0in,34,2,1,3,1,13,0,1,40,1,46,78621


In [48]:
vehicles = feature_df['selected_vehicle'].str.split('_', expand=True)
feature_df['vehicle_type'] = vehicles[0]
feature_df['vehicle_age'] = vehicles[1]
feature_df['vehicle_propulsion'] = vehicles[2]
feature_df = feature_df.drop(['selected_vehicle'], axis=1)

In [49]:
stops = feature_df['stop_frequency'].str.replace(r'[^0-9]', '', regex=True).str.split('', expand=True).drop([0, 3], axis=1)
feature_df['out_stops'] = stops[1]
feature_df['in_stops'] = stops[2]
feature_df = feature_df.drop(['stop_frequency'], axis=1)

In [50]:
zones_dict = zones[['mgra', 'taz']].to_dict('list')
zones_dict_2 = dict(map(lambda i,j: (i,j), zones_dict['mgra'], zones_dict['taz']))
feature_df['destination'] = feature_df['destination'].map(zones_dict_2)
feature_df['origin'] = feature_df['destination'].map(zones_dict_2)

In [51]:
trip_purpose = {'home': 1,
               'shopping': 2,
               'othmaint': 3,
               'work': 4,
               'othdiscr': 5,
               'eatout': 6,
               'escort': 7,
               'atwork': 8,
               'social': 9,
               'school': 10,
               'univ': 11}
feature_df['trip_purpose'] = feature_df['trip_purpose'].map(trip_purpose)

In [52]:
tour_category = {'non_mandatory': 1,
                'mandatory': 2,
                'joint': 3,
                'atwork': 4}
feature_df['tour_category'] = feature_df['tour_category'].map(tour_category)

In [53]:
tour_type = {'work': 1,
             'othdiscr': 2,
             'escort': 3,
             'othmaint': 4,
             'shopping': 5,
             'school': 6,
             'eatout': 7,
             'eat': 8,
             'social': 9,
             'business': 10,
             'maint': 11}
feature_df['tour_type'] = feature_df['tour_type'].map(tour_type)

In [54]:
feature_df['start_time'] = feature_df['start_time'] / 2

In [55]:
feature_df['end_time'] = feature_df['end_time'] / 2

In [56]:
feature_df['duration'] = feature_df['duration'] / 2

In [57]:
vehicle_type = {'Car': 1,
               'SUV': 2,
               'Pickup': 3,
               'Van': 4,
               'Motorcycle': 5,
               'non': np.nan}
feature_df['vehicle_type'] = feature_df['vehicle_type'].map(vehicle_type)

In [58]:
vehicle_propulsion = {'Gas': 1,
                     'Hybrid': 2,
                     'veh': np.nan,
                     'Diesel': 3,
                     'PEV': 4,
                     'BEV': 5}
feature_df['vehicle_propulsion'] = feature_df['vehicle_propulsion'].map(vehicle_propulsion)

In [59]:
true_false = {True: 1,
              False: 2}
feature_df['is_external_tour'] = feature_df['is_external_tour'].map(true_false)
feature_df['is_internal_tour'] = feature_df['is_internal_tour'].map(true_false)

In [60]:
feature_df.head(10)

,trip_id,tour_id,trip_purpose,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,origin,household_id,start_time,end_time,duration,school_esc_outbound,school_esc_inbound,num_escortees,composition,is_external_tour,is_internal_tour,tour_mode,atwork_subtour_frequency,parent_tour_id,age,sex,employment_status,student_status,person_type,education,grade,weeks_worked,hours_worked,race,household_taz,household_income,vehicle_type,vehicle_age,vehicle_propulsion,out_stops,in_stops
0,473,59,7,1,3,3,3,4,4,1,1,64.0,1069.0,1,4.0,4.0,0.0,NaN,NaN,NaN,NaN,2.0,1.0,SHARED3,NaN,NaN,34,2,1,3,1,13,0,1,40,1,46,78621,4.0,6,1.0,0,0
1,477,59,1,1,3,3,3,4,4,1,1,64.0,1069.0,1,4.0,4.0,0.0,NaN,NaN,NaN,NaN,2.0,1.0,SHARED3,NaN,NaN,34,2,1,3,1,13,0,1,40,1,46,78621,4.0,6,1.0,0,0
2,505,63,6,2,7,1,1,1,1,3,5,46.0,1136.0,1,15.0,15.5,0.5,NaN,NaN,NaN,children,2.0,1.0,WALK,NaN,NaN,16,2,3,1,6,0,5,0,0,1,46,78621,NaN,NaN,NaN,0,0
3,509,63,1,2,7,1,1,1,1,3,5,46.0,1136.0,1,15.0,15.5,0.5,NaN,NaN,NaN,children,2.0,1.0,WALK,NaN,NaN,16,2,3,1,6,0,5,0,0,1,46,78621,NaN,NaN,NaN,0,0
4,521,65,5,1,2,1,1,3,4,3,3,46.0,1136.0,1,7.5,8.5,1.0,NaN,NaN,NaN,mixed,2.0,1.0,WALK,NaN,NaN,34,2,1,3,1,13,0,1,40,1,46,78621,NaN,NaN,NaN,0,0
5,525,65,1,1,2,1,1,3,4,3,3,46.0,1136.0,1,7.5,8.5,1.0,NaN,NaN,NaN,mixed,2.0,1.0,WALK,NaN,NaN,34,2,1,3,1,13,0,1,40,1,46,78621,NaN,NaN,NaN,0,0
6,681,85,7,1,3,3,1,1,4,1,1,156.0,2471.0,1,4.5,5.0,0.5,pure_escort,NaN,1.0,NaN,2.0,1.0,SHARED3,NaN,NaN,34,2,1,3,1,13,0,1,40,1,46,78621,4.0,6,1.0,0,0
7,685,85,1,1,3,3,1,1,4,1,1,156.0,2471.0,1,4.5,5.0,0.5,pure_escort,NaN,1.0,NaN,2.0,1.0,SHARED3,NaN,NaN,34,2,1,3,1,13,0,1,40,1,46,78621,4.0,6,1.0,0,0
8,689,86,7,1,3,3,2,2,4,1,1,156.0,2471.0,1,11.5,12.5,1.0,NaN,pure_escort,1.0,NaN,2.0,1.0,SHARED3,NaN,NaN,34,2,1,3,1,13,0,1,40,1,46,78621,4.0,6,1.0,0,0
9,693,86,1,1,3,3,2,2,4,1,1,156.0,2471.0,1,11.5,12.5,1.0,NaN,pure_escort,1.0,NaN,2.0,1.0,SHARED3,NaN,NaN,34,2,1,3,1,13,0,1,40,1,46,78621,4.0,6,1.0,0,0


In [61]:
path = 'C:/Users/RyanS/Jupyter Notebook Assignments/Q2 Project/feature_df.csv'
feature_df.to_csv(path, index=False)